# Importing Dataset

In [10]:
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
df = pl.read_csv('./data/final_dataset.csv')
df.head()

sentiment,review
i64,str
1,"""I have very dry skin and I usu…"
0,"""I own a HP5250.Good side, it i…"
1,"""It has been said that to get w…"
0,"""I didn't purchase this product…"
1,"""I thought that this Tomb Raide…"
…,…
0,"""This is not a very deep album …"
0,"""First off, Amazon is the great…"
1,"""I find it a big large for an s…"


# Preprocessing

In [14]:
import re

In [15]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)